# Knapsack Problem
##### Author: Pamela Bustamante

Problem description: We have $n$ items and a backpack of capacity $V$. Each item $i$ has a given utility $u_i$ and weight $v_i$. We want to choose which items to carry in the backpack, maximizing the obtained utility and without exceeding the capacity of the backpack.

We will define an optimization model to solve this problem.

#### Variables definition

$$ x_i=\left\{
\begin{array}{ll}
      1 & \text{If item i is placed in the backpack} \\
      0 & \text{In other case} \\
\end{array} \; \forall i \in [0,...,n]
\right.  $$

#### Optimization model
\begin{align}
\text{max } & \sum_i u_i x_i \\
\text{s.a} & \\
&\sum_i v_i x_i \le V & \forall i \in [0,...,n]\\
& x_i \in {0,1} & \forall i \in [0,...,n] \\
\end{align}

#### Model in Python + Cplex

In [1]:
import random
import cplex  

We define a function to generate weights and utilities of items

In [2]:
def generate_instance(n):
    #Utilities and weight of items and maximum backpack weight limit are defined. 

    u,v = {}, {}
    for i in range(n):
        u[i] = random.randint(0,100)
        v[i] = random.randint(0,100)

    C = random.uniform(0,1)*sum(v[i] for i in range(n))
    return u,v,C


Se define función para generar modelo

In [3]:
def knapsack(n, V, u, v): 

    #Definition of model.
    model = cplex.Cplex() # We create an object "model".
    model.set_problem_name("Knapsack")
    
    #B = binary
    #obj represents the coeficients of the variables in the objective function
    #obj value by default= 0
    model.variables.add(names=["x[{0}]".format(i) for i in range(n)], types = ['B']*n, obj = [u[i] for i in range(n)])
    
    #The problem is declared to be a maximization problem. By default, the problem is a minimization problem
    model.objective.set_sense(model.objective.sense.maximize)

    #We declare capacity constraints
    coef = [v[i] for i in range(n)]  # coef and index must be of the same dimension, otherwise an error arises
    indices = ["x[{0}]".format(i) for i in range(n)]
    sumq = cplex.SparsePair(ind=indices,val=coef)  #we declare a linear expression
    print(sumq)
    print(V)
    model.linear_constraints.add(lin_expr=[sumq],senses='L',rhs=[V]) # L is less or equal; G greater or equal; E equal
    
    return model


Example of a knapsack problem

In [4]:
#######################
# Example
#######################
    
#We solve the knapsack problem for n objects
n = 10

# We generate a random instance is generated with the previously defined function
u,v,V = generate_instance(n)

#We create a model
mod = knapsack(n, V,u,v)

#We solve a model
mod.solve()

#Results
for i in range(n):
    print("x[{0}]={1}".format(i,mod.solution.get_values("x[{0}]".format(i))))
    
print('Fraction of the objects we carry',len([i for i in range(n) if mod.solution.get_values("x[{0}]".format(i)) == 1])/n)

#Objective function
print('Objective function', mod.solution.get_objective_value())

SparsePair(ind = ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]', 'x[6]', 'x[7]', 'x[8]', 'x[9]'], val = [64, 45, 4, 37, 76, 4, 79, 59, 70, 43])
345.95071560199557
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1 rows, 10 columns, and 10 nonzeros.
Reduced MIP has 10 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 10 columns, and 10 nonzeros.
Reduced MIP has 10 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.00 